In [3]:
import os
import pandas as pd
from Bio import SeqIO
os.chdir('F:\\nanopore_data\\JUfosmid\\seperation\\')

In [229]:
%%bash
wc -l trimmed/flanking_barcode*.list

    7876 trimmed/flanking_barcode01.list
   12730 trimmed/flanking_barcode02.list
    6401 trimmed/flanking_barcode03.list
    8615 trimmed/flanking_barcode04.list
   15125 trimmed/flanking_barcode05.list
   14635 trimmed/flanking_barcode06.list
   10675 trimmed/flanking_barcode07.list
    7514 trimmed/flanking_barcode08.list
   10819 trimmed/flanking_barcode09.list
    8101 trimmed/flanking_barcode10.list
   15591 trimmed/flanking_barcode11.list
    7536 trimmed/flanking_barcode12.list
    5110 trimmed/flanking_barcode13.list
   13463 trimmed/flanking_barcode14.list
    7832 trimmed/flanking_barcode15.list
   16645 trimmed/flanking_barcode16.list
   10367 trimmed/flanking_barcode17.list
   11327 trimmed/flanking_barcode18.list
   11839 trimmed/flanking_barcode19.list
    6885 trimmed/flanking_barcode20.list
    8434 trimmed/flanking_barcode21.list
   10834 trimmed/flanking_barcode22.list
   12010 trimmed/flanking_barcode23.list
   13845 trimmed/flanking_barcode24.list
    9327 trimmed

In [124]:
def fileName(j=13):
    if j <10:
        i = "barcode0"+str(j)
    else:
        i = "barcode"+str(j)
    return(i)
barcoder = fileName(13)

In [125]:
barcoder

'barcode13'

In [126]:
readlist_fh = str("trimmed/flanking_%s.list" % barcoder)
flanking_align_fh = str("trim_paf2csv/filter_%s.csv" % barcoder)
fasta_fh = str("trimmed/trimmed_%s.fa" %barcoder)
output_fh = str("python_fos_sep/fos_%s.csv" %barcoder)

In [127]:
# trimming information file 
flanking_align = pd.read_csv(flanking_align_fh, sep = "\t")

In [128]:
#check read in align information
flanking_align.head()

,qname,qlen,qstart,qend,strand,rname,rlen,rstart,rend,qper,rper
0,9303bac0-b095-42b4-bae2-74ff9926a3ab_L,379,36,246,+,af8edf71-e42c-4cc4-9447-572f7a95e8f2_L,821,54,255,0.554090,0.244823
1,9303bac0-b095-42b4-bae2-74ff9926a3ab_L,379,38,252,+,d79dc385-84c0-47c1-b560-e2203b97d09d_L,357,53,268,0.564644,0.602241
2,9303bac0-b095-42b4-bae2-74ff9926a3ab_L,379,38,231,+,ed620524-c061-4063-b297-b109f7e06fa8_L,838,59,251,0.509235,0.229117
3,9303bac0-b095-42b4-bae2-74ff9926a3ab_L,379,84,252,+,e80f6fec-adcd-4998-8c32-3dd7cdc8c62c_L,341,79,248,0.443272,0.495601
4,9303bac0-b095-42b4-bae2-74ff9926a3ab_L,379,79,248,+,b68476b0-ec8b-44e0-8b6d-9ae1a919ddd8_L,363,96,251,0.445910,0.426997


In [ ]:
#read in fasta for fosmid seperation sequence
record_dict = SeqIO.index(fasta_fh, "fasta")


In [103]:
record_dict.close()

In [130]:
#set cutoff parameter
position_set = 400
cutoff_set = 0.70

In [197]:
#extract df which qname or rname containing read name
def extractAlign(readname):
    single_foslist_initial = list()
    read_align_q = flanking_align[flanking_align["qname"].str.contains(readname)]
    read_align_r = flanking_align[flanking_align["rname"].str.contains(readname)]
    df_read_initial = read_align_q.append(read_align_r)
    #filter fosmid flanking alignment are parallelly aligned
    df_tmp = df_read_initial[((df_read_initial["qname" and "rname"].str.contains("R")) & 
                    (df_read_initial["qstart"] < position_set) & 
                    (df_read_initial["rstart"] < position_set) & 
                    ((df_read_initial["qper"] > cutoff_set)|(df_read_initial["rper"] > cutoff_set))) |
                    ((df_read_initial["qname" and "rname"].str.contains("L")) &
                     (df_read_initial["qlen"] - df_read_initial["qend"] < position_set) &
                     (df_read_initial["rlen"] - df_read_initial["rend"]< position_set) &
                     ((df_read_initial["qper"] > cutoff_set)|(df_read_initial["rper"] > cutoff_set)))]
    #get the initial fosmid sepeartion read list
    if len(df_tmp.index) == 0:
        return(single_foslist_initial)
    else:
        single_fos_list = list(set(list(df_tmp["qname"].str.slice(0,36))+list(df_tmp["rname"].str.slice(0,36))))
        return(single_fos_list)

In [221]:
def looping_foslist(single_fos_list):
    final_foslist = list()
    if len(single_fos_list) != 0:
        final_foslist = single_fos_list
        for read_screening in single_fos_list:
            if read_screening == readname:
                continue
            else:
                single_foslist_screen = extractAlign(read_screening)
                if len(single_foslist_screen) == 0:
                    continue
                else:
                    for screen_element in single_foslist_screen:
                        if screen_element in final_foslist :
                            continue
                        elif screen_element in fos_df[["readname"]]:
                            continue
                        elif len(screen_element)==0:
                            continue
                        else:
                            final_foslist.append(screen_element)
        return(final_foslist)
    else:
        return(single_fos_list)

In [222]:
fos_df = pd.DataFrame(columns=["fosmid", "readname"])

In [223]:
counter = 0
with open(readlist_fh, "r") as idlist:
    with open(output_fh,"w") as fw:
        for line in idlist:
            readname = line.rstrip()
            if readname == "9976a127-2a54-4fb4-a3e6-a0c3843e56f0":
                break
            uniq_list = looping_foslist(extractAlign(readname))
            if len(uniq_list) == 0:
                continue
            else:
                counter +=1
                for list_element in uniq_list:
                    fos_df = fos_df.append(pd.DataFrame([[counter,list_element]],columns=["fosmid", "readname"]))
    fw.close()
idlist.close()

9303bac0-b095-42b4-bae2-74ff9926a3ab working
e80f6fec-adcd-4998-8c32-3dd7cdc8c62c 36
b68476b0-ec8b-44e0-8b6d-9ae1a919ddd8 36
895303f7-b0e3-4cce-a753-02a38b8bdcc2 36
9303bac0-b095-42b4-bae2-74ff9926a3ab 36
d79dc385-84c0-47c1-b560-e2203b97d09d 36
0a088951-84a5-48d5-b3e4-b183e5f48c75 36
5de5c0d4-30c3-4124-a952-96782c133263 36
6bd561ca-9cb8-4f5c-b7af-1228d6f143c0 36
755cac91-0b84-4c68-9320-959b013befbf 36
e80f6fec-adcd-4998-8c32-3dd7cdc8c62c 36
9303bac0-b095-42b4-bae2-74ff9926a3ab 36
895303f7-b0e3-4cce-a753-02a38b8bdcc2 36
b68476b0-ec8b-44e0-8b6d-9ae1a919ddd8 36
d79dc385-84c0-47c1-b560-e2203b97d09d 36
0a088951-84a5-48d5-b3e4-b183e5f48c75 36
5de5c0d4-30c3-4124-a952-96782c133263 36
e80f6fec-adcd-4998-8c32-3dd7cdc8c62c 36
9303bac0-b095-42b4-bae2-74ff9926a3ab 36
895303f7-b0e3-4cce-a753-02a38b8bdcc2 36
b68476b0-ec8b-44e0-8b6d-9ae1a919ddd8 36
d79dc385-84c0-47c1-b560-e2203b97d09d 36
0a088951-84a5-48d5-b3e4-b183e5f48c75 36
5de5c0d4-30c3-4124-a952-96782c133263 36
6bd561ca-9cb8-4f5c-b7af-1228d6f143c

dc483149-8dd8-41ff-bd18-16ddc59fb496 36
d7a60056-5eda-4cb2-ada0-15d1b0f676b1 36
d90461f2-5a35-4e8d-891b-8ec39c8f929e 36
e238400c-75dd-448a-bac5-87cc10e3d859 36
45307894-7cbd-476c-8f9d-371c17629c93 36
97015462-f4b7-4196-98d5-d76d25dfe53e 36
e2c5bce5-f10d-4ffa-b40d-c27ba0f3e212 36
97780787-6db5-4246-b4ee-1dad3e727ed3 36
f69a85d0-fa01-4a21-a384-72515045e50f 36
89e9c2be-56c3-44f1-8075-30ce01cce82b 36
01781b8c-d55a-46dc-a10d-52bc10cd874e 36
9b0a9b7c-a4bd-4a5a-9890-413484ea5aee 36
1107ca53-5d0b-4128-a79b-2a1e0c0a0d80 36
91b7087a-ffdd-44fa-b823-f03033da53fc 36
ad12d445-ca39-4640-bab4-80c3b1a9678b 36
5b499109-4353-4d33-916d-89ac04e1818c 36
726d2171-ea8d-41d0-8dcb-30e9ed6704ec 36
04c915ef-84f8-4fab-8d48-1f684df822d2 36
ad12d445-ca39-4640-bab4-80c3b1a9678b 36
726d2171-ea8d-41d0-8dcb-30e9ed6704ec 36
04c915ef-84f8-4fab-8d48-1f684df822d2 36
5b499109-4353-4d33-916d-89ac04e1818c 36
14597043-84a2-444e-a2a8-b0391dec3da1 36
e4602ddc-ac27-4f10-967f-2a6e2d89774e 36
e238400c-75dd-448a-bac5-87cc10e3d859 36


dc483149-8dd8-41ff-bd18-16ddc59fb496 36
d90461f2-5a35-4e8d-891b-8ec39c8f929e 36
45307894-7cbd-476c-8f9d-371c17629c93 36
6db417b3-c47e-420b-8517-5ea40afaad4d 36
89e9c2be-56c3-44f1-8075-30ce01cce82b 36
e2c5bce5-f10d-4ffa-b40d-c27ba0f3e212 36
6fe86bb5-97ad-4368-b639-02dcfd45cf74 36
dc483149-8dd8-41ff-bd18-16ddc59fb496 36
d90461f2-5a35-4e8d-891b-8ec39c8f929e 36
45307894-7cbd-476c-8f9d-371c17629c93 36
6db417b3-c47e-420b-8517-5ea40afaad4d 36
89e9c2be-56c3-44f1-8075-30ce01cce82b 36
e2c5bce5-f10d-4ffa-b40d-c27ba0f3e212 36
6fe86bb5-97ad-4368-b639-02dcfd45cf74 36
91b7087a-ffdd-44fa-b823-f03033da53fc 36


In [115]:
fos_df.append(pd.DataFrame([[1,readname]],columns=["fosmid", "readname"]))

,fosmid,readname
0,1,44b73770-d1c6-40f4-979b-8e069e21c2a8


In [116]:
uniq_list = looping_foslist(extractAlign(readname))

In [120]:
for list_element in uniq_list:
    fos_df=fos_df.append(pd.DataFrame([[1,list_element]],columns=["fosmid", "readname"]))

In [121]:
fos_df

,fosmid,readname
0,1,213f1b1a-ef9f-4aee-a464-a858895bc0d0
0,1,220f5026-50e6-426a-a49c-7f479dfb737b
0,1,44b73770-d1c6-40f4-979b-8e069e21c2a8
0,1,79e40f4d-3eee-416b-a513-9c5b0d7f4af7
0,1,2e78c769-443b-469d-844a-66a03434bf50
0,1,2f0e3275-eb12-4472-b067-f0ae3404cab4
0,1,2c3d1690-80b3-4b82-b587-12a2b65f0be9
0,1,c8c83903-f2e6-4fcb-8f6d-a70ced945f95
